## 라이브러리 설치

In [ ]:
!pip install xlsxwriter

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.0/153.0 kB 2.7 MB/s eta 0:00:00


In [ ]:
pip install pmdarima


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 20.9 MB/s eta 0:00:00


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import itertools
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
%matplotlib inline
import statsmodels.api as sm

from datetime import datetime, timedelta

from dateutil.relativedelta import relativedelta
from warnings import filterwarnings
filterwarnings('ignore')
import os
import itertools
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
%matplotlib inline
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error
from datetime import datetime
from dateutil.relativedelta import relativedelta
from warnings import filterwarnings
filterwarnings('ignore')
import os
from sklearn.preprocessing import MinMaxScaler

In [ ]:
RRP.head(30)

NameError: ignored

## 데이터 로드 및 데이터 전처리

In [3]:
RRP = pd.read_excel('/content/drive/MyDrive/new_ABC.xlsx')

In [4]:
co_df=RRP.copy()

In [5]:
co_df.set_index('Requested_Date',inplace=True)

In [6]:
scaler = MinMaxScaler()
co_df[['Requested_Quantity']] = scaler.fit_transform(co_df[['Requested_Quantity']])

In [7]:
co_df

,Requested_Quantity,PRODNAME
Requested_Date,,
2018-11-08,0.012459,AE
2018-11-08,0.186882,CSA5000
2018-11-08,0.093441,PEMA-580FX
2018-11-08,0.940641,PEMA-HR1000
2018-11-08,0.211800,PEMA-HR1500
...,...,...
2021-04-24,0.000000,PEMA-SP1000
2021-04-24,0.000000,SRE-200
2021-04-24,0.000000,PEMA-SPR


## train, test 분할

In [8]:
std_date = co_df.index[-1]
testdate_start = std_date - relativedelta(months=6)
traindate_end = testdate_start - relativedelta(days=1)
traindate_start = co_df.index[0]

## 모델학습 및 파라미터 설정 및 검증 결과 출력

In [12]:
predicted_values_dic = {}
pdq_li = []
r2_li = []
aic_li = []
period = []
r2_remove0_li = []

for i in range(1):
  print('train  :',traindate_start.date(),'~',traindate_end.date())
  print('test   :',testdate_start.date(),'~',std_date.date())
  r2_li1 = []
  r2_remove0_li1 = []
  aic_li1 = []
  pdq_li1 = []
  period.append(testdate_start.strftime('%y%m%d')+'~'+std_date.strftime('%y%m%d'))

  for prod in co_df['PRODNAME'].unique():
    print(f'<<<<<<<<<<<<<<<  {prod}({testdate_start.date()} ~ {std_date.date()})  >>>>>>>>>>>>>>>')
    series = co_df.query(f'PRODNAME=="{prod}"')
    series.drop('PRODNAME',axis=1,inplace=True)
    series = series.resample('W').sum()

    # 데이터 분리

    train = series['Requested_Quantity'][traindate_start:traindate_end]
    test = series['Requested_Quantity'][testdate_start:std_date]


    # Grid_search

    p = d = q = range(0,2)
    pdq = list(itertools.product(p,d,q))
    seasonal_pdq = [(x[0],x[1],x[2],12) for x in list(itertools.product(p,d,q))]
    best_aic = np.inf
    best_pdq = None
    best_seasonal_pdq = None
    tmp_model = None
    best_mdl = None

    for param in pdq:
      for param_seasonal in seasonal_pdq:
        try:
          tmp_mdl = sm.tsa.statespace.SARIMAX(train, exog=None, order=param,
                                              seasonal_order = param_seasonal,
                                              enforce_stationarity=True,
                                              enforce_invertibility=True)
          res = tmp_mdl.fit()
          if res.aic < best_aic:
            best_aic = res.aic
            best_pdq = param
            best_seasonal_pdq = param_seasonal
            best_mdl = tmp_mdl
        except:
          continue
    print("Best SARIMAX{}x{}12 model - AIC : {}".format(best_pdq, best_seasonal_pdq, best_aic))

    # 학습

    res = sm.tsa.statespace.SARIMAX(endog=train, order=best_pdq,
                                seasonal_order = best_seasonal_pdq,
                                enforce_stationarity=True,
                                enforce_invertibility=True).fit()

    aic_li1.append(res.aic)
    pdq_li1.append((best_pdq, best_seasonal_pdq, best_aic))
    display(res.summary())

    try:
      res.plot_diagnostics(figsize=(16,10))
    except:
      pass

    # 시각화

    pred = res.get_prediction(start=test.index.min(),
                            end = test.index.max(),
                            dynamic=True)

    pred_ci = pred.conf_int()

    plt.figure(figsize=(14,7))
    ax = series[traindate_start:std_date].plot(label = 'Observed',color='#006699');
    pred.predicted_mean.plot(ax=ax, label='Prediction', alpha=.7, color='#ff0066');

    ax.fill_between(pred_ci.index,
                    pred_ci.iloc[:,0],
                    pred_ci.iloc[:,1],color='#ff0066', alpha=.25);

    ax.fill_betweenx(ax.get_ylim(), test.index.min(),
                    test.index.max(), alpha=.15, zorder=-1,
                    color='gray')
    ax.set_xlabel('Time (week)')
    ax.set_ylabel(f'weekly SOLD_QUANT({prod})')
    ax.set_title('Test data vs prediction By SARIMA')
    plt.legend(loc='upper left')
    plt.show()

    # 검증(R2_score)
    from sklearn.metrics import mean_squared_error
    from sklearn.metrics import r2_score
    predicted_value = pred.predicted_mean
    r2 = r2_score(test,predicted_value)
    r2_li1.append(r2)
    print('R2_score :',r2)
    print()
    mse = mean_squared_error(test, predicted_value)
    print(f'MSE: {mse}')
    print()
    rmse = np.sqrt(mse)
    print(f'RMSE: {rmse}')
    print()
    mape = np.mean(np.abs((test - predicted_value) / test)) * 100
    print('MAPE:', mape)
    print()

    # 모델, 결과값 저장

    res.save(f'/content/drive/MyDrive/국취제/{prod}{i}.pkl')
    try:
      df = pd.concat([predicted_value,test],axis=1)
      df.to_excel(f'/content/drive/MyDrive/국취제/{prod}{i}.xlsx')
      predicted_values_dic[f'{prod}{i+1}'] = df
      df1 = df[df['Requested_Quantity']!=0]
      r2_remove0_li1.append(r2_score(df1['Requested_Quantity'],df1[0]))
    except:
      predicted_values_dic[f'{prod}{i}'] = predicted_value

  print('='*80)
  std_date = testdate_start - relativedelta(weeks=1)
  testdate_start = std_date - relativedelta(months=6)
  traindate_end = testdate_start - relativedelta(days=1)
  traindate_start = (traindate_end - relativedelta(years=2, months=3))
  r2_li.append(r2_li1)
  aic_li.append(aic_li1)
  pdq_li.append(pdq_li1)
  r2_remove0_li.append(r2_remove0_li1)




# # 결과 저장

# r2 = pd.DataFrame(r2_li)
# r2.columns = co_df['PRODNAME'].unique()
# r2 = r2.T
# r2.columns = period

# aic = pd.DataFrame(aic_li)
# aic.columns = co_df['PRODNAME'].unique()
# aic = aic.T
# aic.columns = period

# pdq = pd.DataFrame(pdq_li)
# pdq.columns = co_df['PRODNAME'].unique()
# pdq = pdq.T
# pdq.columns = period

# r2_remove0 = pd.DataFrame(r2_remove0_li)
# r2_remove0.columns = co_df['PRODNAME'].unique()
# r2_remove0 = r2_remove0.T
# r2_remove0.columns = period

# results = pd.DataFrame()
# path = '/content/drive/MyDrive/sarima_train/국취제'
# file_list = os.listdir(path)
# for file in file_list:
#   df = pd.read_excel(path + '/' + file)
#   prod_name = file[:-5]
#   df['Prod_name'] = prod_name
#   results = pd.concat([results, df])
# results = results[results['Requested_Quantity']!=0] # 마이너스 값 삭제(Requested_Quantity 0일 때 예측나온 값들)
# results.columns=['Time','Predicted_Quantity','Requested_Quantity','Prod_name']
# results = results.set_index('Time').sort_index()

# writer = pd.ExcelWriter(f'/content/drive/MyDrive/국취제/results.xlsx',engine='xlsxwriter')
# results.to_excel(writer, sheet_name='results')
# r2.to_excel(writer, sheet_name='r2')
# aic.to_excel(writer, sheet_name='aic')
# pdq.to_excel(writer, sheet_name='pdq')
# r2_remove0.to_excel(writer, sheet_name='r2_remove0')

# writer.save()

Output hidden; open in https://colab.research.google.com to view.

## 모델 불러와서 제품별 예측량 뽑기

In [10]:
import pandas as pd
import pickle

# 리스트에 있는 모든 제품들을 정의합니다.
products = ['AE', 'CSA5000', 'PEMA-580FX', 'PEMA-HR1000', 'PEMA-HR1500',
            'PEMA-PR1000', 'PEMA-SR2000', 'PEMA-SR3000F', 'CSA4000',
            'PEMA-500FR', 'PEMA-CR1000', 'PEMA-CSA5000', 'PEMA-SN400',
            'PEMA-SR2000A', 'PEMA-PCM2000', 'PEMA-PCM2000B', 'PEMA-PR2000',
            'CSA4000(PCA)', 'PR1000', 'PEMA-PCR3000E', 'PEMA-SR5000F',
            'PEMA-PCR3000N', 'PEMA-SP1000', 'SRE-200', 'PEMA-SPR',
            'PEMA-HR1000S', 'SRE-110']
product_dataframes = {}
for product in products:
    # 모델 불러오기
    with open(f'/content/drive/MyDrive/국취제/{product}0.pkl', 'rb') as file:
        loaded_model = pickle.load(file)

    forecast_steps = 56
    forecast = loaded_model.get_forecast(steps=forecast_steps)
    forecast_values = forecast.predicted_mean
    df = pd.DataFrame(forecast_values, columns=['predicted_mean'])
    df['week'] = df.index - pd.Timedelta(days=6)
    df = df.set_index('week')

    df.rename(columns={'predicted_mean': '예측중량'}, inplace=True)
    start_date = pd.to_datetime('2021-04-19')
    end_date = pd.to_datetime('2021-07-26')
    filtered_df = df[(df.index >= start_date) & (df.index <= end_date)]

    # 정규화된 데이터에서 최소값과 최대값을 구합니다.
    min_val = RRP['Requested_Quantity'].min()
    max_val = RRP['Requested_Quantity'].max()

    # 정규화된 값을 원본 범위로 스케일링합니다.
    filtered_df['예측중량'] = (filtered_df['예측중량'] * (max_val - min_val)) + min_val
    product_dataframes[product] = filtered_df
    # # CSV 파일로 저장합니다.
    filtered_df.to_csv(f'/content/drive/MyDrive/국취제/{product}.csv')


In [ ]:
product_dataframes

{'AE':                     예측중량
 week                    
 2021-04-19  13923.281936
 2021-04-26  11907.333913
 2021-05-03  32401.466866
 2021-05-10  22691.837537
 2021-05-17  14904.428773
 2021-05-24   8048.568329
 2021-05-31  13903.569682
 2021-06-07  23980.122279
 2021-06-14  12413.419201
 2021-06-21  10610.165596
 2021-06-28  28524.109707
 2021-07-05  19268.253712
 2021-07-12  11887.814774
 2021-07-19   6652.798102
 2021-07-26  11766.981395,
 'CSA5000':                      예측중량
 week                     
 2021-04-19  147982.538262
 2021-04-26  253940.323718
 2021-05-03  238740.302422
 2021-05-10  295768.319196
 2021-05-17  213886.516008
 2021-05-24  259680.745317
 2021-05-31  235555.368595
 2021-06-07  241360.298174
 2021-06-14  154716.109843
 2021-06-21  225966.211813
 2021-06-28  210979.960766
 2021-07-05  248418.629817
 2021-07-12  176489.794210
 2021-07-19  228253.274266
 2021-07-26  212437.234236,
 'PEMA-580FX':                      예측중량
 week                     
 2021-04-19 

In [ ]:
product_dataframes

{'AE':                     예측중량
 week                    
 2021-04-26  11907.333913
 2021-05-03  32401.466866
 2021-05-10  22691.837537
 2021-05-17  14904.428773
 2021-05-24   8048.568329
 2021-05-31  13903.569682
 2021-06-07  23980.122279
 2021-06-14  12413.419201
 2021-06-21  10610.165596
 2021-06-28  28524.109707
 2021-07-05  19268.253712
 2021-07-12  11887.814774
 2021-07-19   6652.798102
 2021-07-26  11766.981395,
 'CSA5000':                      예측중량
 week                     
 2021-04-26  253940.323718
 2021-05-03  238740.302422
 2021-05-10  295768.319196
 2021-05-17  213886.516008
 2021-05-24  259680.745317
 2021-05-31  235555.368595
 2021-06-07  241360.298174
 2021-06-14  154716.109843
 2021-06-21  225966.211813
 2021-06-28  210979.960766
 2021-07-05  248418.629817
 2021-07-12  176489.794210
 2021-07-19  228253.274266
 2021-07-26  212437.234236,
 'PEMA-580FX':                      예측중량
 week                     
 2021-04-26   66302.418828
 2021-05-03  134951.920044
 2021-05-10

In [ ]:
product_dataframes

{'AE':                     예측중량       week prodname
 week                                        
 2021-04-26  11833.763760 2021-04-26       AE
 2021-05-03  32486.403244 2021-05-03       AE
 2021-05-10  22605.431866 2021-05-10       AE
 2021-05-17  14860.941450 2021-05-17       AE
 2021-05-24   7972.781105 2021-05-24       AE
 2021-05-31  13735.053251 2021-05-31       AE
 2021-06-07  23984.260616 2021-06-07       AE
 2021-06-14  12382.843818 2021-06-14       AE
 2021-06-21  10622.485660 2021-06-21       AE
 2021-06-28  28632.328971 2021-06-28       AE
 2021-07-05  19181.044176 2021-07-05       AE
 2021-07-12  11855.341531 2021-07-12       AE
 2021-07-19   6594.661695 2021-07-19       AE
 2021-07-26  11574.786249 2021-07-26       AE,
 'CSA5000':                      예측중량       week prodname
 week                                         
 2021-04-26  253943.140842 2021-04-26  CSA5000
 2021-05-03  238742.936254 2021-05-03  CSA5000
 2021-05-10  295772.484361 2021-05-10  CSA5000
 2021-05-17

In [ ]:
product_dfs = []
for product, product_df in product_dataframes.items():
    product_df['week'] = product_df.index  # 인덱스를 week 컬럼으로 추가
    product_df['prodname'] = product  # prodname 컬럼 추가
    product_dfs.append(product_df)

all_products_df = pd.concat(product_dfs, ignore_index=True)
all_products_df = all_products_df.set_index('week')
all_products_df = all_products_df.sort_index()
auto_df=all_products_df
auto_df = auto_df.reset_index()

In [ ]:
auto_df['온도'] = None
auto_df['습도'] = None
auto_df['sold_quant'] = None
auto_df['건설기성액'] = None
auto_df['국내건설수주액'] = None
new_column_order = ['week', 'prodname', '온도', '습도','예측중량','sold_quant', '건설기성액', '국내건설수주액']
auto_df = auto_df[new_column_order]
auto_df.rename(columns={'예측중량':'order_quant'}, inplace=True)

In [ ]:
auto_df

,week,prodname,온도,습도,order_quant,sold_quant,건설기성액,국내건설수주액
0,2021-04-19,AE,None,None,1.392328e+04,None,None,None
1,2021-04-19,PEMA-SN400,None,None,2.530781e+04,None,None,None
2,2021-04-19,PEMA-PCM2000B,None,None,4.894586e+03,None,None,None
3,2021-04-19,PEMA-SPR,None,None,-8.323122e-15,None,None,None
4,2021-04-19,PEMA-HR1000,None,None,4.166721e+05,None,None,None
...,...,...,...,...,...,...,...,...
400,2021-07-26,CSA4000,None,None,2.342456e+04,None,None,None
401,2021-07-26,PEMA-PCM2000B,None,None,7.356849e+03,None,None,None
402,2021-07-26,PEMA-CR1000,None,None,-1.901906e+03,None,None,None
403,2021-07-26,PEMA-SR2000,None,None,7.646794e+04,None,None,None
